In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
cookies = {'session':'53616c7465645f5ff4b35a6eb924fde728d21409670086f7994aae3d5f27a69035f4466a65096035e2fe25fc34f889af'}

def get_data(qno):
    response = requests.get(f'https://adventofcode.com/2020/day/{str(qno)}/input', cookies = cookies)
    data = response.text
    return data

# Q1. 
Specifically, they need you to find the two entries that sum to 2020 and then multiply those two numbers together.

In [ ]:
# response

# response.headers

In [4]:
from itertools import combinations

a, b = [pair for pair in combinations(data, 2) if sum(pair) == 2020][0]
print(a * b)

326211


In your expense report, what is the product of the three entries that sum to 2020?

In [6]:
a, b, c = [triple for triple in combinations(data, 3) if sum(triple) == 2020][0]
print(a * b * c)

131347190


# Q2. 


In [5]:
data = get_data(2)
data = data.split('\n')
# data

import re
valid = 0
for rule in data[:-1]:
    bounds, char, _, pwd = [x.strip() for x in re.split(':| ', rule)]
    lb, ub = [int(x) for x in bounds.split('-')] 
    count = list(pwd).count(char)
    if  count <= ub and count >= lb:
        valid += 1
print(valid)

600


Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

In [6]:
valid = 0
for i, rule in enumerate(data[:-1]):
    pos, char, _, pwd = [x.strip() for x in re.split(':| ', rule)]
    p1, p2 = [int(x)-1 for x in pos.split('-')] 
    if int(pwd[p1] == char) + int(pwd[p2] == char) == 1:
        valid += 1
print(valid)

245


# Q3. 
Starting at the top-left corner of your map and following a slope of right 3 and down 1, how many trees would you encounter?

In [7]:
data = get_data(3)
data = data.strip()

import numpy as np

n = len(data.split('\n'))
data = np.array(list(data.replace('\n','')))
data = data.reshape(n,-1)

count, i, j = 0, 0, 0 
d2 = data[:]

while i < data.shape[0]:
    if d2[i, j] == '#':
        count += 1
    i += 1
    j += 3
    if j >= d2.shape[1]:
        d2 = np.concatenate((d2, data), axis = 1)
        
print(count)

262


Determine the number of trees you would encounter if, for each of the following slopes, you start at the top-left corner and traverse the map all the way to the bottom:

1. Right 1, down 1.
2. Right 3, down 1. (This is the slope you already checked.)
3. Right 5, down 1.
4. Right 7, down 1.
5. Right 1, down 2.

What do you get if you multiply together the number of trees encountered on each of the listed slopes?

In [8]:
ans = 1
slopes = [(1, 1),(3, 1), (5, 1), (7, 1), (1, 2)] # right and down
d2 = ''

for t in slopes:
    count, i, j = 0, 0, 0
    d2 = data[:]
    while i < data.shape[0]:
        if d2[i, j] == '#':
            count += 1
        j += t[0]
        i += t[1]

        if j >= data.shape[1]:
            d2 = np.concatenate((d2, data), axis = 1)
    
    ans *= count
        
print(ans)

2698900776
